In [2]:
from skorecard import datasets
from skorecard.bucketers import DecisionTreeBucketer, OptimalBucketer
from skorecard.pipeline import make_coarse_classing_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

df = datasets.load_uci_credit_card(as_frame=True)
X = df.drop(columns=["default"])
y = df["default"]

num_cols = ["LIMIT_BAL", "BILL_AMT1"]
cat_cols = ["EDUCATION", "MARRIAGE"]

pipeline = make_pipeline(
    DecisionTreeBucketer(variables=num_cols, max_n_bins=100, min_bin_size=0.05),
    make_coarse_classing_pipeline(
        OptimalBucketer(variables=num_cols, max_n_bins=10, min_bin_size=0.05),
        OptimalBucketer(variables=cat_cols, max_n_bins=10, min_bin_size=0.05),
    ),
    OneHotEncoder(),
    LogisticRegression(),
)

pipeline.fit(X, y)

from skorecard.apps import BucketTweakerApp
tweaker = BucketTweakerApp(pipeline, X, y)

# Three modes to try: ['inline','external', 'jupyterlab']
# https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
tweaker.run_server(mode='inline')
tweaker.stop_server()